# Importing Data

In [1]:
#Imports for the codebase
import pandas as pd 
import datetime as dt

In [3]:
#Reading in the csvs and saving them to ny_total_df and sf_total_df
ny_total_df = pd.read_csv('./Resources/NYPD_Arrests_Data__Historic__20231221.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Resources/NYPD_Arrests_Data__Historic__20231221.csv'

In [ ]:
##making sure that the data was read properly
display(ny_total_df.head())

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,236791704,11/22/2021,581.0,NaN,NaN,NaN,PL 2225001,M,M,28,0.0,45-64,M,BLACK,997427.0,230378.0,40.799009,-73.952409,POINT (-73.95240854099995 40.799008797000056)
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
3,32311380,06/18/2007,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,1.0,18-24,M,BLACK,NaN,NaN,NaN,NaN,NaN
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)


In [ ]:
#Displaying the columns and saving it to a value
ny_crimes_df = ny_total_df['OFNS_DESC'].value_counts()
# sf_crimes_df = sf_total_df['Category']

display(ny_crimes_df.head(25))
# display(sf_crimes_df.tail(20))

OFNS_DESC
DANGEROUS DRUGS                                1109839
ASSAULT 3 & RELATED OFFENSES                    573150
OTHER OFFENSES RELATED TO THEFT                 304191
PETIT LARCENY                                   255208
FELONY ASSAULT                                  244707
OTHER STATE LAWS                                242211
VEHICLE AND TRAFFIC LAWS                        217351
DANGEROUS WEAPONS                               212781
MISCELLANEOUS PENAL LAW                         208778
CRIMINAL TRESPASS                               203882
ROBBERY                                         179078
OTHER TRAFFIC INFRACTION                        178868
OFFENSES AGAINST PUBLIC ADMINISTRATION          156544
POSSESSION OF STOLEN PROPERTY 5                 149877
CRIMINAL MISCHIEF & RELATED OFFENSES            146039
GRAND LARCENY                                   141677
INTOXICATED & IMPAIRED DRIVING                  106403
FORGERY                                          91263


# Cleaning data

In [ ]:
ny_total_df = ny_total_df.dropna()

In [ ]:
# Converting the data types to correct values
ny_total_df['ARREST_DATE'] = pd.to_datetime(ny_total_df['ARREST_DATE'], format='%m/%d/%Y')

In [ ]:
#sanity check
ny_total_df.dtypes

ARREST_KEY                    int64
ARREST_DATE          datetime64[ns]
PD_CD                       float64
PD_DESC                      object
KY_CD                       float64
OFNS_DESC                    object
LAW_CODE                     object
LAW_CAT_CD                   object
ARREST_BORO                  object
ARREST_PRECINCT               int64
JURISDICTION_CODE           float64
AGE_GROUP                    object
PERP_SEX                     object
PERP_RACE                    object
X_COORD_CD                  float64
Y_COORD_CD                  float64
Latitude                    float64
Longitude                   float64
Lon_Lat                      object
dtype: object

# Dropping unnecessary columns

## columns:
ARREST_DATE -> date

PD_DESC -> description

OFNS_DESC -> category

PD_CD -> incident_code

ARREST_BORO -> pd_district

 

Create dayofweek column

In [ ]:
ny_cleaned_df = ny_total_df[['ARREST_DATE', 'PD_DESC', 'OFNS_DESC', 'PD_CD', 'ARREST_BORO']]
ny_cleaned_df['dayofweek'] = ny_cleaned_df['ARREST_DATE'].dt.day_name()

ny_cleaned_df = ny_cleaned_df.rename(columns={'ARREST_DATE':'date', 'PD_DESC':'description', 'OFNS_DESC':'category', 'PD_CD':'incident_code', 'ARREST_BORO':'pd_district'})

ny_cleaned_df.head()


/var/folders/q0/kyk7n6qs3cj9s3l8y_40h1rr0000gn/T/ipykernel_58663/995903816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_cleaned_df['dayofweek'] = ny_cleaned_df['ARREST_DATE'].dt.day_name()


,date,description,category,incident_code,pd_district,dayofweek
1,2021-12-04,RAPE 3,RAPE,153.0,B,Saturday
2,2021-11-09,"CHILD, ENDANGERING WELFARE",SEX CRIMES,681.0,Q,Tuesday
4,2019-01-26,SEXUAL ABUSE,SEX CRIMES,177.0,M,Saturday
7,2021-11-10,"ARSON 2,3,4",ARSON,263.0,B,Wednesday
8,2021-12-28,"FORGERY,ETC.,UNCLASSIFIED-FELO",FORGERY,729.0,Q,Tuesday


# Sorting the data

In [ ]:
#Sorting by dangerous drugs
ny_drug_df = ny_cleaned_df.loc[ny_cleaned_df['category'].str.contains('DRUGS', na=False)]
ny_drug_df.tail()

,date,description,category,incident_code,pd_district,dayofweek
5498593,2022-12-29,"CONTROLLED SUBSTANCE,SALE 3",DANGEROUS DRUGS,515.0,M,Thursday
5498598,2022-12-07,"CONTROLLED SUBSTANCE, POSSESSI",DANGEROUS DRUGS,511.0,B,Wednesday
5498616,2022-12-29,"CONTROLLED SUBSTANCE, POSSESSI",DANGEROUS DRUGS,511.0,B,Thursday
5498628,2022-12-09,"CONTROLLED SUBSTANCE, POSSESSI",DANGEROUS DRUGS,511.0,B,Friday
5498636,2022-12-11,"CONTROLLED SUBSTANCE,INTENT TO",DANGEROUS DRUGS,503.0,K,Sunday


In [ ]:
#sorting drugs by dates in 2008
ny_drugs_dec_2008 = ny_drug_df.loc[(ny_drug_df['date'] >= '2008-12-01') & (ny_drug_df['date'] <= '2008-12-31')].set_index('date')
ny_drugs_dec_2008.head()

,description,category,incident_code,pd_district,dayofweek
date,,,,,
2008-12-10,"CONTROLLED SUBSTANCE,SALE 3",DANGEROUS DRUGS,515.0,K,Wednesday
2008-12-15,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,M,Monday
2008-12-13,"CONTROLLED SUBSTANCE,SALE 3",DANGEROUS DRUGS,515.0,M,Saturday
2008-12-12,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,M,Friday
2008-12-06,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,M,Saturday


In [ ]:
#sorting drugs by dates in 2017
ny_drugs_dec_2017 = ny_drug_df.loc[(ny_drug_df['date'] >= '2017-12-01') & (ny_drug_df['date'] <= '2017-12-31')].set_index('date')
ny_drugs_dec_2017.head()

,description,category,incident_code,pd_district,dayofweek
date,,,,,
2017-12-02,"CONTROLLED SUBSTANCE,SALE 3",DANGEROUS DRUGS,515.0,B,Saturday
2017-12-07,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,B,Thursday
2017-12-01,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,B,Friday
2017-12-22,SALE SCHOOL GROUNDS,DANGEROUS DRUGS,523.0,S,Friday
2017-12-12,"MARIJUANA, POSSESSION 4 & 5",DANGEROUS DRUGS,567.0,K,Tuesday


In [ ]:
#Sorting by prostitution
ny_prostitution_df = ny_cleaned_df.loc[ny_cleaned_df['category'].str.contains('PROSTITUTION', na=False)]
ny_prostitution_df.head()

,date,description,category,incident_code,pd_district,dayofweek
40,2018-10-26,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,Q,Friday
875,2020-11-06,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,B,Friday
1042,2020-12-30,"PROSTITUTION 4,PROMOTING&SECUR",PROSTITUTION & RELATED OFFENSES,576.0,B,Wednesday
1308,2020-12-23,SEX TRAFFICKING,PROSTITUTION & RELATED OFFENSES,586.0,B,Wednesday
1664,2020-11-06,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,B,Friday


In [ ]:
#sorting prostitution by dates in 2008
ny_prostitution_dec_2008 = ny_prostitution_df.loc[(ny_prostitution_df['date'] >= '2008-12-01') & (ny_prostitution_df['date'] <= '2008-12-31')].set_index('date')
ny_prostitution_dec_2008.head()

,description,category,incident_code,pd_district,dayofweek
date,,,,,
2008-12-31,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,B,Wednesday
2008-12-30,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,Q,Tuesday
2008-12-31,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,B,Wednesday
2008-12-30,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,B,Tuesday
2008-12-31,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,B,Wednesday


In [ ]:
#sorting prostitution by dates in 2017
ny_prostitution_dec_2017 = ny_prostitution_df.loc[(ny_prostitution_df['date'] >= '2017-12-01') & (ny_prostitution_df['date'] <= '2017-12-31')].set_index('date')
ny_prostitution_dec_2017.head()

,description,category,incident_code,pd_district,dayofweek
date,,,,,
2017-12-02,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,B,Saturday
2017-12-08,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,K,Friday
2017-12-16,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,K,Saturday
2017-12-13,PROSTITUTION,PROSTITUTION & RELATED OFFENSES,563.0,Q,Wednesday
2017-12-08,"PROSTITUTION, PATRONIZING 4, 3",PROSTITUTION & RELATED OFFENSES,565.0,K,Friday


# Saving data to csv

In [ ]:
#Reset indexes in dataframes
ny_drugs_dec_2008 = ny_drugs_dec_2008.reset_index()
ny_drugs_dec_2017 = ny_drugs_dec_2017.reset_index()

ny_prostitution_dec_2008 = ny_prostitution_dec_2008.reset_index()
ny_prostitution_dec_2017 = ny_prostitution_dec_2017.reset_index()

In [ ]:
#save to CSV file
ny_drugs_dec_2008.to_csv("./Cleaned_Resources/ny_drugs_dec_2008.csv", index=False)
ny_drugs_dec_2017.to_csv("./Cleaned_Resources/ny_drugs_dec_2017.csv", index=False)

ny_prostitution_dec_2008.to_csv("./Cleaned_Resources/ny_prostitution_dec_2008.csv", index=False)
ny_prostitution_dec_2017.to_csv("./Cleaned_Resources/ny_prostitution_dec_2017.csv", index=False)